In [8]:
import pandas as pd

In [2]:
df_raw = pd.read_csv("GT_trajectories_raw.csv")
print(df_raw.shape)
df_raw.head()

(623, 10)


,gt_id,shortage_event_id,ndc_id,Status,Timestamp,Reason for Shortage,Generic Name,Company Name,Initial Posting Date,Date of Update
0,5,00527-1395-11/25/2019,00527-1395,to be discontinued,8/14/23 0:01,NaN,Loxapine Capsules,"Lannett Company, Inc.",11/25/19,6/14/22
1,5,00527-1396-11/25/2019,00527-1396,to be discontinued,8/14/23 0:01,NaN,Loxapine Capsules,"Lannett Company, Inc.",11/25/19,6/14/22
2,5,00527-1397-11/25/2019,00527-1397,to be discontinued,8/14/23 0:01,NaN,Loxapine Capsules,"Lannett Company, Inc.",11/25/19,6/14/22
3,5,00591-0371-11/25/2019,00591-0371,resolved,8/14/23 0:01,NaN,Loxapine Capsules,Teva Pharmaceuticals,11/25/19,6/14/22
4,5,00527-1394-11/25/2019,00527-1394,to be discontinued,8/14/23 0:01,NaN,Loxapine Capsules,"Lannett Company, Inc.",11/25/19,6/14/22


In [3]:
df_raw.columns

Index(['gt_id', 'shortage_event_id', 'ndc_id', 'Status', 'Timestamp',
       ' Reason for Shortage', 'Generic Name', 'Company Name',
       'Initial Posting Date', 'Date of Update'],
      dtype='object')

In [4]:
df_raw['Initial Posting Date'] = pd.to_datetime(df_raw['Initial Posting Date'])
df_raw['Date of Update'] = pd.to_datetime(df_raw['Date of Update'])

df_raw['quarters_between'] = (
    df_raw['Date of Update'].dt.to_period('Q')         
    - df_raw['Initial Posting Date'].dt.to_period('Q')
).apply(lambda x: x.n)


/var/folders/33/gyvvd4710r7_4n22mnftfz4h0000gn/T/ipykernel_3734/94494800.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_raw['Initial Posting Date'] = pd.to_datetime(df_raw['Initial Posting Date'])
/var/folders/33/gyvvd4710r7_4n22mnftfz4h0000gn/T/ipykernel_3734/94494800.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_raw['Date of Update'] = pd.to_datetime(df_raw['Date of Update'])


In [5]:
df_raw['company_code'] = df_raw['ndc_id'].apply(lambda x: x.split("-")[0])
df_raw

,gt_id,shortage_event_id,ndc_id,Status,Timestamp,Reason for Shortage,Generic Name,Company Name,Initial Posting Date,Date of Update,quarters_between,company_code
0,5,00527-1395-11/25/2019,00527-1395,to be discontinued,8/14/23 0:01,NaN,Loxapine Capsules,"Lannett Company, Inc.",2019-11-25,2022-06-14,10,00527
1,5,00527-1396-11/25/2019,00527-1396,to be discontinued,8/14/23 0:01,NaN,Loxapine Capsules,"Lannett Company, Inc.",2019-11-25,2022-06-14,10,00527
2,5,00527-1397-11/25/2019,00527-1397,to be discontinued,8/14/23 0:01,NaN,Loxapine Capsules,"Lannett Company, Inc.",2019-11-25,2022-06-14,10,00527
3,5,00591-0371-11/25/2019,00591-0371,resolved,8/14/23 0:01,NaN,Loxapine Capsules,Teva Pharmaceuticals,2019-11-25,2022-06-14,10,00591
4,5,00527-1394-11/25/2019,00527-1394,to be discontinued,8/14/23 0:01,NaN,Loxapine Capsules,"Lannett Company, Inc.",2019-11-25,2022-06-14,10,00527
...,...,...,...,...,...,...,...,...,...,...,...,...
618,80,49884-0156-06/24/2021,49884-0156,resolved,8/14/23 0:01,NaN,Varenicline Tartrate (Chantix) Tablets,Par Pharmaceutical,2021-06-24,2022-05-26,4,49884
619,80,49884-0944-06/24/2021,49884-0944,resolved,8/14/23 0:01,NaN,Varenicline Tartrate (Chantix) Tablets,Par Pharmaceutical,2021-06-24,2022-05-26,4,49884
620,80,60505-4766-06/24/2021,60505-4766,resolved,8/14/23 0:01,NaN,Varenicline Tartrate (Chantix) Tablets,Apotex Corp.,2021-06-24,2022-05-26,4,60505
621,80,60505-4765-06/24/2021,60505-4765,resolved,8/14/23 0:01,NaN,Varenicline Tartrate (Chantix) Tablets,Apotex Corp.,2021-06-24,2022-05-26,4,60505


In [6]:
def summarize(group: pd.DataFrame) -> pd.Series:
    # manufacturers that have at least one “to be discontinued” row
    disc_rows     = group['Status'].eq('to be discontinued')
    disc_makers   = group.loc[disc_rows, 'Company Name'].unique()

    # --- build the usual columns ---
    n_makers      = group['Company Name'].nunique()
    gt_type       = group['Status'].nunique()
    quarters        = group['quarters_between'].iloc[0]          # first value in original order
    disruption_nr = len(disc_makers)

    # --- disruption magnitude (average percentage across those makers) ---
    if disruption_nr == 0:          # no disruptions in this gt_id
        disruption_mag = 0.2
    else:
        # proportion of “discontinued” rows for each disrupted maker
        maker_pct = (
            group[group['Company Name'].isin(disc_makers)]
            .groupby('Company Name')['Status']
            .apply(lambda s: (s == 'to be discontinued').mean())   # mean() on booleans → proportion
        )
        disruption_mag = maker_pct.mean()        # average across makers

    return pd.Series({
        'n_manufacturers'   : n_makers,
        'gt_type'           : gt_type,
        'quarters'            : quarters,
        'disruption_number' : disruption_nr,
        'disruption_magnitude': disruption_mag     # 0 – 1 proportion; multiply by 100 for %
    })

# one tidy summary table
df = df_raw.groupby('gt_id', as_index=False).apply(summarize)
df['disruption_magnitude'] = df.disruption_magnitude.apply(lambda x: 0.6 if x > 0.5 else x)
df['periods'] = df.quarters.apply(lambda x: 4 if x <4 else x)

/var/folders/33/gyvvd4710r7_4n22mnftfz4h0000gn/T/ipykernel_3734/2567830033.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df_raw.groupby('gt_id', as_index=False).apply(summarize)


In [7]:
df.gt_type.value_counts()

gt_type
1.0    57
2.0    16
Name: count, dtype: int64

In [ ]:
# enforced disruption start
print(df.loc[(df.periods < 12) & (df.gt_type == 2)].shape)
df.loc[(df.periods < 12) & (df.gt_type == 2)].to_csv("GT_Disc.csv", index=False)

(11, 7)


In [113]:
# random disruption start
print(df.loc[(df.periods < 12) & (df.gt_type == 1)].shape)
df.loc[(df.periods < 12) & (df.gt_type == 1)].to_csv("GT_NoDisc.csv", index=False)

(41, 7)


# Evaluation

In [65]:
import ast
gt_disc_file_name = "gt_experiments_20250730_220734.csv"
df = pd.read_csv("../src/gt_evaluation/" + gt_disc_file_name)
df.trajectory = df.trajectory.apply(lambda x: ast.literal_eval(x))

In [67]:
df.n_manufacturers.mean()

np.float64(4.090909090909091)

In [68]:
df.periods.mean()

np.float64(6.0)

In [19]:
tmp = df.explode('trajectory', ignore_index=True)
sim_cols = (
    pd.json_normalize(tmp['trajectory'])  # dict → columns
      .rename(lambda c: f"trajectory_{c}", axis=1)  # optional: prefix to avoid clashes
)
final_df = (
    tmp.drop(columns='trajectory')   # keep every other column once
       .reset_index(drop=True)        # tidy up the index after explode
       .join(sim_cols)                # add the flattened columns
)

In [21]:
final_df.columns

Index(['scenario', 'simulation_id', '#simulation', 'gt_type',
       'n_manufacturers', 'periods', 'disruption_prob', 'disruption_magnitude',
       'peak_shortage', 'avg_shortage', 'shortage_periods', 'resolution_time',
       'total_profit', 'buyer_cost', 'fda_interventions', 'trajectory_period',
       'trajectory_total_demand', 'trajectory_total_supply',
       'trajectory_shortage_amount', 'trajectory_unsold',
       'trajectory_shortage_percentage', 'trajectory_disrupted_manufacturers',
       'trajectory_fda_announcement'],
      dtype='object')

In [34]:
final_df['shortage_flag'] = final_df.apply(lambda x: abs(x.trajectory_total_supply - 1) < 0.05, axis=1)
# final_df.to_csv("current.csv")

In [35]:
cols_to_sort = ['scenario', 'trajectory_period']
df_sorted = final_df.sort_values(cols_to_sort)

# get the last row for every (scenario, simulation) combo
final_flag = (
    df_sorted
      .groupby(['scenario'])
      .tail(1)['shortage_flag']
)

# resolution success (=1 if shortage_actual == False)
resolved = (~final_flag).astype(int)

resolution_success_rate = resolved.mean()
print(f"Resolution Success Rate: {resolution_success_rate:.2%}")

Resolution Success Rate: 72.73%


In [ ]:
t_sim = (
    df_sorted.loc[df_sorted.shortage_flag]               # rows where shortage == False
          .groupby(['scenario'])   # one per run
          .agg(t_sim=('trajectory_period', 'min'))
)
t_sim = t_sim + 1

In [55]:
t_gt = (
    df_sorted.groupby('scenario')['trajectory_period']      # or ['periods'] if you prefer
         .max()                              # horizon == resolved quarter
         .rename('t_gt')
)
t_gt = t_gt+1

In [49]:
horizon = (df_sorted.groupby(['scenario'])
                   .trajectory_period.max())

In [58]:
meta = (
    t_sim.join(horizon.rename('T'), how='left')
         .reset_index()
         .merge(t_gt.reset_index(), on='scenario', how='left')
)

meta['t_sim'] = meta['t_sim'].fillna(meta['T'] + 1)  # unresolved sims

In [59]:
###############################################################################
# 3.  Lag & Resolution-Lag %  (unchanged)
###############################################################################
meta['lag'] = meta['t_sim'] - meta['t_gt']
meta['rlp'] = 100 * meta['lag'] / meta['t_gt']

overall_rlp_mean = meta['rlp'].mean()
print(f"Mean Resolution-Lag %: {overall_rlp_mean:6.2f}")

Mean Resolution-Lag %: -53.00


In [61]:
###############################################################################
# 1. Simulation: sustainable resolution time  (last True + 1)
###############################################################################
# last period that is still in shortage
t_last_short = (
    df_sorted.loc[df_sorted.shortage_flag]                  # rows where shortage persists
          .groupby(['scenario'])
          .agg(t_last_true=('trajectory_period', 'max'))
)

# add 1 → first period with no shortage *and stays that way thereafter*
t_sim = t_last_short.assign(t_sim=lambda x: x['t_last_true'] + 1)[['t_sim']]

###############################################################################
# Ground-truth horizon is still the resolution time
# (if your GT stops exactly when shortage ends)
###############################################################################
t_gt = (
    df_sorted.groupby('scenario')['trajectory_period']
         .max()
         .rename('t_gt')
)

###############################################################################
# 2-3. Continue exactly as before: join → fill unresolved →
#      lag = t_sim − t_gt,  rlp = 100·lag / t_gt
###############################################################################
horizon = (df_sorted.groupby(['scenario'])
                   .trajectory_period.max())

meta = (
    t_sim.join(horizon.rename('T'), how='left')
         .reset_index()
         .merge(t_gt.reset_index(), on='scenario', how='left')
)

# unresolved sims: no shortage_flag == False at all  → t_sim = T + 1
meta['t_sim'] = meta['t_sim'].fillna(meta['T'] + 1)

meta['lag'] = meta['t_sim'] - meta['t_gt']
meta['rlp'] = 100 * meta['lag'] / meta['t_gt']

In [63]:
meta['rlp'].mean()

np.float64(9.444444444444446)

In [64]:
t_sim

,t_sim
scenario,
gt_id_14,4
gt_id_20,5
gt_id_5,10
gt_id_61,4
gt_id_66,7
